# 7.1 엑셀 업무 자동화_실습
  
## 1. 실습_유통매장의 고객만족도 분석하기   

### 1) 엑셀 파일 불러오기 
* load_workbook모듈과 고객 만족 데이터.xlsx 파일 불러오기

In [1]:
from openpyxl import load_workbook
wb=load_workbook("고객 만족 데이터.xlsx")
ws=wb["Sheet1"]

### 2) 데이터 변환하기   
* 반복문을 활용해 [A2:F13] 범위를 정수형으로 변환  

In [2]:
for row in ws:
    for cell in row: 
        if cell.col_idx>0 and cell.col_idx<7and cell.row>1:
            cell.data_type="i"

### 3) 데이터 요약하기   
* B, C, D는 평균을 구함  
* E, F는 합계를 구함

In [7]:
this_max_row=ws.max_row
ws["A{}".format(this_max_row+1)]="평균/합계"
for col_let in ["B","C","D"]:
    average_formula = "=AVERAGE({}2:{}{})".format(col_let, col_let, this_max_row)
    ws["{}{}".format(col_let, this_max_row+1)]=average_formula
    ws["{}{}".format(col_let, this_max_row+1)].number_format="#.##"

for col_let in ["E","F"]:
    sum_formula="=SUM({}2:{}{})".format(col_let, col_let, this_max_row)
    ws["{}{}".format(col_let, this_max_row+1)]=sum_formula

### 4) 요약표 작성하기   
* create_sheet()함수로 시트를 하나 생성한 후, ws객체 생성 
- 요약표 형식을 우선 만든 후, 요약표에 들어갈 값을 구함

In [8]:
wb.create_sheet("보고서 삽입용 요약표")
ws=wb["보고서 삽입용 요약표"]

summary_table_form=[["성별 분포","남성","여성"], ["인원","",""],["비율","",""],["","",""],
                    ["연령분포","20대","30대","40대","50대 이상"], ["인원","",""],["비율","",""],
                   ["","",""],["조사 결과"],["제품 만족도"],["재구매 의사"],["추천 의향"],
                   ["방문 횟수"],["클레임 횟수"]]
for form_row in summary_table_form:
    ws.append(form_row)

* 1) 반복문 
    - 남/녀 인원수와 비율 구함  
* 2) 반복문
    - 연령대별 인원수와 비율을 구함
* 3) 반복문  
    - 첫번째 시트의 B15, C15, D15 셀의 평균값, E15, F15셀의 합계를 B10:B14로 복사 

In [12]:
for gender in [["B","남"],["C","여"]]:
    ws[gender[0]+"2"]="=COUNTIF(Sheet1!H2:H{},\"{}\")".format(ws.max_row, gender[1])
    ws[gender[0]+"3"]="={}2/SUM(B2:C2)".format(gender[0])
    ws[gender[0]+"3"].number_format="0.00%"
    
    for age in [["B","20대"],["C","30대"],["D","40대"],["E","50대 이상"]]:
        ws[age[0]+"6"]="=COUNTIF(Sheet1!G2:G{},\"{}\")".format(ws.max_row, age[1])
        ws[age[0]+"7"]="={}6/SUM(B6:E6)".format(age[0])
        ws[age[0]+"7"].number_format="0.00%"
        
    row_no=10
    for alphabet in ["B","C","D","E","F"]:
        ws["B"+str(row_no)]="=Sheet1!{}{}".format(alphabet, ws.max_row+1)
        if row_no<13:
            ws["B"+str(row_no)].number_format="#.##"
        row_no +=1

### 5) 서식 설정하기   
* 셀 너비와 높이 변경  
    - range(len(summary_table_form)+1)는 요약표 양식이 있는 행의 길이 반환 

In [13]:
ws.column_dimensions["A"].width=15
for alphabet in ["B","C","D","F"]:
    ws.column_dimensions[alphabet].width=10

for i in range(len(summary_table_form)+1):
    ws.row_dimensions[i].height=25

In [18]:
from openpyxl.styles import Alignment, Side, Border, Font, PatternFill, Color
alignment_style = Alignment(horizontal="center", vertical="center")

side=Side(color="CCCCCC", border_style="medium")
border_style=Border(
    left=side,
    right=side,
    top=side,
    bottom=side,)

style_apply_cells=["A1:C3", "A5:E7", "A9:B14"]
for cell_area in style_apply_cells:
    for row in ws[cell_area]:
        for cell in row:
            cell.alignment=alignment_style
            cell.border=border_style

In [20]:
header_font_style = Font(sz=12, bold=True)
header_background_style=PatternFill(fill_type="solid", start_color="ffd663")
style_apply_cells_for_header=["A1:C1","A5:E5","A9:B9"]

for cell_area in style_apply_cells_for_header:
    for row in ws[cell_area]:
        for cell in row:
            cell.font=header_font_style
            cell.fill=header_background_style
            
ws.merge_cells("A9:B9")

* 엑셀 활성화 시트를 요약표 시트로 변경  
    - sb객체를 엑셀파일로 저장

In [21]:
wb.active=ws
wb.save("고객 만족 데이터_보고.xlsx")

## 2. 실습_여러 개의 엑셀 파일 하나로 합치기  

### 1) 모듈 불러오기

In [22]:
from copy import copy
from openpyxl import Workbook  
from openpyxl import load_workbook